In [ ]:
# python3 -m venv elbaff_iesta_venv --prompt="elbaff_iesta_venv"
# source elbaff_iesta_venv/bin/activate

In [1]:
from nlpaf.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np
import sys, os
import importlib

import iesta.loader as loader
import iesta.properties as prop  
import iesta.processor as proc  
import iesta.feature_extractor as fe
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY

from collections import Counter




In [ ]:
conservative_each_dataloader = IESTAData(ideology=prop.CONSERVATIVE_IDEOLOGY, methodology=METHODOLOGY.EACH)





In [ ]:
conservative_each_dataloader.load()

In [ ]:
conservative_each_dataloader.pivot_df

In [ ]:
conservative_each_dataloader.pivot_binary_effect

In [ ]:
conservative_training_each_df  = conservative_each_dataloader.data_df[conservative_each_dataloader.data_df["split"] == "training"].copy()
len(conservative_training_each_df)

In [ ]:
conservative_training_each_df.columns.tolist()

In [2]:
import dataclasses
class TransformerBasedFeaturePipeline(Pipeline):
    
    def __init__(self,
                 input= None,
                 load_default_pipe_configs = True,
                 extended_pipe_configs:dict = None,
                 save_output= False,
                 out_path = None,
                 argument_col:str = "argument"
                 ):
        super().__init__(input,
                 load_default_pipe_configs,
                 extended_pipe_configs,
                 save_output,
                 out_path)
        self.argument_col= argument_col
        
    def process_input(self) -> list:
        processed = []
        txt_df = self.input[["id", self.argument_col]].copy()
        txt_df =txt_df.rename(columns={"id": "input_id", self.argument_col: "text"},)
        
        for idx, row in txt_df.iterrows():
            processed.append((row.text, {"input_id": row.input_id}))

        return processed

    def init_and_run(self):
        
        self.add_annotation_pipe(name = "sentencizer", save_output= False,is_spacy=True, is_native=True)
        self.add_annotation_pipe(name = "EmotionPipeOrchestrator", save_output= True, is_spacy=True)
        #self.add_annotation_pipe(name = "HedgePipeOrchestrator",   save_output= True, is_spacy=True)
        #self.add_annotation_pipe(name = "ToxicityOrchestrator",    save_output= True, is_spacy=True)

        # annotate the input
        #self.set_spacy_language_model("en_core_web_md")
        self.annotate()
        # save annotations when "save_output" is set to True
        #self.save()

In [ ]:
pipeline.nlp

In [5]:
import pandas  as pd
test_df = pd.DataFrame([{"id": 1, "argument": "This is a very hard time, I am devastated! This a happy sentence 2."},
                         {"id": 2, "argument": "You can not get your tiny brain to work on this so stupid!!"},
                         {"id": 3, "argument": "The amonium Nitrate was sitting there for ages."},
                         {"id": 4, "argument": "I love you and I love how you look"}
                       ])


test_df

,id,argument
0,1,"This is a very hard time, I am devastated! Thi..."
1,2,You can not get your tiny brain to work on thi...
2,3,The amonium Nitrate was sitting there for ages.
3,4,I love you and I love how you look


In [6]:
from nlpaf.util.timer import Timer

pipeline = TransformerBasedFeaturePipeline(save_output= False,
                                          out_path = "features_test.parquet")
pipeline.set_input(test_df)
#pipeline.set_input(test_df)
t = Timer()
t.start()
pipeline.init_and_run()
t.stop()
#pipeline.out_df.head()

nlpaf        INFO     adding pipe with name EmotionPipeOrchestrator
nlpaf        DEBUG    loading nlpaf.annotator.pipe.linguistic.emotion
nlpaf        INFO     orchestrator was initialized successfully
nlpaf        INFO     adding pipe with code emotion_hartmann_component
nlpaf        INFO     Defining pipe default and spacy stacks


get_pipe_stacks Elapsed time: 0.0009 seconds
get_pipe_stacks Elapsed time: 0.0009 seconds
init emotion_hartmann_component


nlpaf        INFO     Pipes are ['sentencizer', 'emotion_hartmann_component']


 Elapsed time: 28.1898 seconds
 Elapsed time: 28.1898 seconds
 Elapsed time: 82.7342 seconds
 Elapsed time: 82.7342 seconds


82.73421969999998

In [ ]:
pipeline = TransformerBasedFeaturePipeline(save_output= True,
                 out_path = "../data/extracted_features/features_conservative_training_each.parquet")
pipeline.set_input(conservative_training_each_df)
#pipeline.set_input(test_df)
pipeline.init_and_run()
pipeline.out_df.head()

In [ ]:
lib_transformers_df = pd.read_parquet("../data/extracted_features/features_liberal_training_each.parquet")

lib_transformers_df.head()
lib_transformers_df['hedge_dominant'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['emotion_hartmann_label'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['toxicity_dominant'].value_counts().to_frame().plot(kind="bar")

lib_transformers_df[["emotion_hartmann_anger",
                     "emotion_hartmann_disgust",
                    "emotion_hartmann_fear", 
                    "emotion_hartmann_joy",
                    "emotion_hartmann_neutral",
                    "emotion_hartmann_surprise",
                    "emotion_hartmann_sadness"]].plot.hist(bins=12, alpha=0.5)